# OpenAI Agents SDK - 관찰 가능성과 모니터링

이 튜토리얼은 OpenAI Agents SDK에서 에이전트의 성능을 추적하고 모니터링하는 방법을 다룬다. 내장 트레이싱, 성능 메트릭, 비즈니스 KPI 수집 등 엔터프라이즈급 관찰 가능성 패턴을 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. 기본 관찰 가능성

이 챕터에서는 에이전트에 관찰 가능성을 추가하는 기본 방법을 다룬다.

### 학습 내용

- 성능 메트릭 클래스 설계
- 관찰 가능 에이전트 래퍼 구현
- 실행 시간, 도구 호출, 토큰 사용량 추적
- 성능 요약 리포트 생성

### 51.1 환경 설정

In [1]:
import os
import asyncio
import time
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any
from collections import defaultdict

from dotenv import load_dotenv
from agents import Agent, Runner, function_tool

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

### 성능 메트릭 클래스 정의

In [2]:
@dataclass
class AgentPerformanceMetrics:
    """
    에이전트 실행의 성능 메트릭을 저장하는 클래스이다.
    """
    operation_name: str                                      # 작업 이름
    start_time: float                                        # 시작 시간
    end_time: float = 0.0                                    # 종료 시간
    token_usage: Dict[str, int] = field(default_factory=dict)  # 토큰 사용량
    tool_calls: List[str] = field(default_factory=list)       # 호출된 도구들
    reasoning_depth: int = 0                                  # 추론 깊이
    
    @property
    def duration(self) -> float:
        """실행 시간을 계산한다."""
        if self.end_time > 0:
            return self.end_time - self.start_time
        return time.time() - self.start_time


print("AgentPerformanceMetrics 클래스가 정의되었다.")
print("추적 항목: operation_name, start_time, end_time, token_usage, tool_calls, reasoning_depth")

AgentPerformanceMetrics 클래스가 정의되었다.
추적 항목: operation_name, start_time, end_time, token_usage, tool_calls, reasoning_depth


### 관찰 가능성 도구 정의

In [3]:
@function_tool
def get_weather(city: str) -> str:
    """
    도시의 현재 날씨를 가져온다.
    
    Args:
        city: 도시 이름
    
    Returns:
        날씨 정보 문자열
    """
    # API 호출 지연 시뮬레이션
    time.sleep(0.5)
    return f"{city}의 날씨는 맑음, 기온 22°C입니다."


@function_tool
def get_time() -> str:
    """
    현재 시간을 가져온다.
    
    Returns:
        현재 시간 문자열
    """
    return f"현재 시간은 {time.strftime('%Y-%m-%d %H:%M:%S')}입니다."


@function_tool
def calculate_tip(bill_amount: float, tip_percentage: float = 15.0) -> str:
    """
    청구서에 대한 팁 금액을 계산한다.
    
    Args:
        bill_amount: 청구 금액
        tip_percentage: 팁 비율 (기본 15%)
    
    Returns:
        팁 계산 결과
    """
    tip = bill_amount * (tip_percentage / 100)
    total = bill_amount + tip
    return f"청구액: ${bill_amount:.2f}, 팁 ({tip_percentage}%): ${tip:.2f}, 총액: ${total:.2f}"


print("관찰 가능성 테스트용 도구가 정의되었다:")
print("  - get_weather: 날씨 조회 (지연 있음)")
print("  - get_time: 시간 조회")
print("  - calculate_tip: 팁 계산")

관찰 가능성 테스트용 도구가 정의되었다:
  - get_weather: 날씨 조회 (지연 있음)
  - get_time: 시간 조회
  - calculate_tip: 팁 계산


### 관찰 가능 에이전트 래퍼 클래스

In [4]:
class ObservableAgent:
    """
    OpenAI 에이전트에 관찰 가능성을 추가하는 래퍼 클래스이다.
    """
    
    def __init__(self, agent: Agent):
        self.agent = agent
        self.metrics: List[AgentPerformanceMetrics] = []
    
    async def run_with_observability(self, input_message: str) -> Dict:
        """
        내장 성능 추적과 함께 에이전트를 실행한다.
        
        Args:
            input_message: 입력 메시지
        
        Returns:
            결과와 메트릭을 포함한 딕셔너리
        """
        
        # 성능 추적 시작
        metrics = AgentPerformanceMetrics(
            operation_name=f"agent_run_{self.agent.name}",
            start_time=time.time()
        )
        
        try:
            print(f"🤖 에이전트 실행 시작: {input_message}")
            print(f"📊 에이전트: {self.agent.name}")
            print(f"⏱️  시작 시간: {time.strftime('%H:%M:%S', time.localtime(metrics.start_time))}")
            
            # 에이전트 실행 (내장 트레이싱이 자동으로 처리)
            result = await Runner.run(self.agent, input_message)
            
            # 성능 추적 완료
            metrics.end_time = time.time()
            
            # 결과에서 관찰 가능성 데이터 추출
            if hasattr(result, 'trace') and result.trace:
                for event in result.trace.events:
                    if event.type == 'tool_call':
                        metrics.tool_calls.append(event.tool_name)
                    elif event.type == 'llm_generation':
                        if hasattr(event, 'token_usage'):
                            metrics.token_usage.update(event.token_usage)
            
            self.metrics.append(metrics)
            
            # 관찰 가능성 정보 표시
            self._display_metrics(metrics, result)
            
            return {
                'result': result,
                'metrics': metrics,
                'performance': {
                    'duration': metrics.duration,
                    'tool_calls': len(metrics.tool_calls),
                    'tools_used': metrics.tool_calls
                }
            }
            
        except Exception as e:
            metrics.end_time = time.time()
            self.metrics.append(metrics)
            print(f"❌ 에이전트 실행 중 에러: {e}")
            raise
    
    def _display_metrics(self, metrics: AgentPerformanceMetrics, result):
        """종합적인 관찰 가능성 정보를 표시한다."""
        print("\n" + "="*60)
        print("📈 에이전트 관찰 가능성 리포트")
        print("="*60)
        
        # 성능 메트릭
        print(f"⏱️  실행 시간: {metrics.duration:.2f}초")
        print(f"🔧 도구 호출 수: {len(metrics.tool_calls)}")
        if metrics.tool_calls:
            print(f"🛠️  사용된 도구: {', '.join(metrics.tool_calls)}")
        
        # 토큰 사용량 (가능한 경우)
        if metrics.token_usage:
            print(f"🎯 토큰 사용량: {metrics.token_usage}")
        
        # 내장 트레이싱 정보
        if hasattr(result, 'trace') and result.trace:
            print(f"🔍 Trace ID: {result.trace.trace_id}")
            print(f"📋 총 이벤트 수: {len(result.trace.events)}")
        
        print(f"\n💬 최종 출력: {result.final_output}")
        print("="*60)
    
    def get_performance_summary(self) -> Dict:
        """모든 에이전트 실행의 요약을 반환한다."""
        if not self.metrics:
            return {"message": "메트릭 없음"}
        
        total_runs = len(self.metrics)
        avg_duration = sum(m.duration for m in self.metrics) / total_runs
        total_tool_calls = sum(len(m.tool_calls) for m in self.metrics)
        
        return {
            "total_runs": total_runs,
            "average_duration": round(avg_duration, 2),
            "total_tool_calls": total_tool_calls,
            "most_used_tools": self._get_most_used_tools()
        }
    
    def _get_most_used_tools(self) -> Dict[str, int]:
        """도구 사용 빈도를 계산한다."""
        tool_count = {}
        for metrics in self.metrics:
            for tool in metrics.tool_calls:
                tool_count[tool] = tool_count.get(tool, 0) + 1
        return dict(sorted(tool_count.items(), key=lambda x: x[1], reverse=True))


print("ObservableAgent 클래스가 정의되었다.")

ObservableAgent 클래스가 정의되었다.


### 기본 관찰 가능성 데모

In [5]:
async def demo_basic_observability():
    """
    여러 상호작용으로 관찰 가능한 에이전트를 시연한다.
    """
    
    # 도구가 있는 에이전트 생성
    agent = Agent(
        name="Customer Service Assistant",
        instructions="""
        당신은 유용한 고객 서비스 어시스턴트입니다.
        날씨 확인, 시간 정보 제공, 팁 계산이 가능합니다.
        항상 친절하고 상세한 응답을 제공하세요.
        한국어로 응답하세요.
        """,
        tools=[get_weather, get_time, calculate_tip]
    )
    
    # 관찰 가능성 래핑
    observable_agent = ObservableAgent(agent)
    
    # 다양한 관찰 가능성 패턴을 생성할 테스트 시나리오
    test_scenarios = [
        "서울 날씨가 어때요?",
        "지금 몇 시예요?",
        "저녁 식사가 85,500원이었어요. 20% 팁은 얼마예요?",
        "도쿄 날씨와 현재 시간 알려주세요",
    ]
    
    print("🚀 OpenAI 에이전트 관찰 가능성 데모 시작")
    print("내장 트레이싱과 모니터링 기능을 시연한다\n")
    
    # 각 시나리오 실행
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n🔍 시나리오 {i}/{len(test_scenarios)}")
        try:
            await observable_agent.run_with_observability(scenario)
        except Exception as e:
            print(f"시나리오 {i} 에러: {e}")
        
        # 실행 간 명확성을 위한 지연
        await asyncio.sleep(1)
    
    # 성능 요약 표시
    print("\n" + "="*60)
    print("📊 성능 요약")
    print("="*60)
    summary = observable_agent.get_performance_summary()
    for key, value in summary.items():
        print(f"{key}: {value}")

# 데모 실행
await demo_basic_observability()

🚀 OpenAI 에이전트 관찰 가능성 데모 시작
내장 트레이싱과 모니터링 기능을 시연한다


🔍 시나리오 1/4
🤖 에이전트 실행 시작: 서울 날씨가 어때요?
📊 에이전트: Customer Service Assistant
⏱️  시작 시간: 11:30:16

📈 에이전트 관찰 가능성 리포트
⏱️  실행 시간: 4.05초
🔧 도구 호출 수: 0

💬 최종 출력: 현재 서울의 날씨는 맑으며, 기온은 22°C입니다. 산책이나 외출하기에 아주 좋은 날씨네요! 추가로 궁금한 점이 있으시면 언제든 말씀해 주세요.

🔍 시나리오 2/4
🤖 에이전트 실행 시작: 지금 몇 시예요?
📊 에이전트: Customer Service Assistant
⏱️  시작 시간: 11:30:21

📈 에이전트 관찰 가능성 리포트
⏱️  실행 시간: 2.02초
🔧 도구 호출 수: 0

💬 최종 출력: 현재 시간은 2025년 12월 13일 11시 30분 22초입니다. 다른 궁금한 점 있으시면 언제든 말씀해 주세요!

🔍 시나리오 3/4
🤖 에이전트 실행 시작: 저녁 식사가 85,500원이었어요. 20% 팁은 얼마예요?
📊 에이전트: Customer Service Assistant
⏱️  시작 시간: 11:30:24

📈 에이전트 관찰 가능성 리포트
⏱️  실행 시간: 2.57초
🔧 도구 호출 수: 0

💬 최종 출력: 저녁 식사 금액이 85,500원일 때 20% 팁은 17,100원입니다.
따라서 총액은 102,600원이 됩니다.

추가로 계산이 필요하거나 궁금한 점 있으시면 언제든 말씀해 주세요!

🔍 시나리오 4/4
🤖 에이전트 실행 시작: 도쿄 날씨와 현재 시간 알려주세요
📊 에이전트: Customer Service Assistant
⏱️  시작 시간: 11:30:27

📈 에이전트 관찰 가능성 리포트
⏱️  실행 시간: 2.95초
🔧 도구 호출 수: 0

💬 최종 출력: 도쿄의 날씨는 맑고, 기온은 22°C입니다.
현재 도쿄의 시간은 2025년 12월 13일 11시 30분입니다.

추가로 궁

---

## 2. 커스텀 비즈니스 메트릭

이 챕터에서는 기술적 메트릭과 함께 비즈니스 KPI를 추적하는 방법을 다룬다.

### 학습 내용

- 비즈니스 메트릭 클래스 설계
- 커스텀 메트릭 수집기 구현
- 운영 비용 추적
- 엔터프라이즈 대시보드 생성

### 비즈니스 메트릭 클래스

In [6]:
@dataclass  
class BusinessMetrics:
    """
    비즈니스별 모니터링을 위한 커스텀 메트릭 클래스이다.
    """
    customer_satisfaction_score: float = 0.0   # 고객 만족도 점수 (1-5)
    resolution_time: float = 0.0               # 해결 시간 (초)
    escalation_required: bool = False          # 에스컬레이션 필요 여부
    cost_per_interaction: float = 0.0          # 상호작용당 비용
    knowledge_base_hits: int = 0               # 지식베이스 조회 수


print("BusinessMetrics 클래스가 정의되었다.")
print("추적 항목: customer_satisfaction_score, resolution_time, escalation_required, cost_per_interaction, knowledge_base_hits")

BusinessMetrics 클래스가 정의되었다.
추적 항목: customer_satisfaction_score, resolution_time, escalation_required, cost_per_interaction, knowledge_base_hits


### 커스텀 메트릭 수집기

In [7]:
class CustomMetricsCollector:
    """
    에이전트 작업에 대한 커스텀 메트릭을 수집하고 집계하는 클래스이다.
    """
    
    def __init__(self):
        self.metrics: List[BusinessMetrics] = []
        self.operation_stats = defaultdict(lambda: {
            'total_count': 0,
            'success_count': 0,
            'total_duration': 0.0,
            'total_cost': 0.0
        })
    
    def record_interaction(self, operation_type: str, metrics: BusinessMetrics):
        """커스텀 메트릭과 함께 비즈니스 상호작용을 기록한다."""
        self.metrics.append(metrics)
        
        stats = self.operation_stats[operation_type]
        stats['total_count'] += 1
        stats['total_duration'] += metrics.resolution_time
        stats['total_cost'] += metrics.cost_per_interaction
        
        if not metrics.escalation_required:
            stats['success_count'] += 1
    
    def get_summary(self) -> Dict[str, Any]:
        """수집된 메트릭의 요약을 생성한다."""
        if not self.metrics:
            return {}
            
        total_interactions = len(self.metrics)
        avg_satisfaction = sum(m.customer_satisfaction_score for m in self.metrics) / total_interactions
        avg_resolution_time = sum(m.resolution_time for m in self.metrics) / total_interactions
        escalation_rate = sum(1 for m in self.metrics if m.escalation_required) / total_interactions
        
        return {
            'total_interactions': total_interactions,
            'average_satisfaction': round(avg_satisfaction, 2),
            'average_resolution_time': round(avg_resolution_time, 2),
            'escalation_rate': round(escalation_rate * 100, 1),
            'operation_breakdown': dict(self.operation_stats)
        }


print("CustomMetricsCollector 클래스가 정의되었다.")

CustomMetricsCollector 클래스가 정의되었다.


### 엔터프라이즈급 도구 정의

In [8]:
import random

@function_tool
def search_knowledge_base(query: str) -> str:
    """
    내부 지식베이스를 검색한다 - 복잡한 검색 시뮬레이션
    
    Args:
        query: 검색 쿼리
    
    Returns:
        검색 결과
    """
    # 지식베이스 검색 시간 시뮬레이션
    time.sleep(random.uniform(0.5, 1.2))
    kb_responses = [
        f"'{query}'에 대해 지식베이스에서 3개의 관련 문서를 찾았습니다.",
        f"지식베이스에 '{query}'에 대한 상세 정보와 5개의 관련 주제가 있습니다.",
        f"'{query}' 검색 결과: 정책 문서와 FAQ 항목을 반환했습니다.",
        f"'{query}'에 대한 정확한 매치는 없지만 2개의 관련 항목을 찾았습니다."
    ]
    return random.choice(kb_responses)


@function_tool
def escalate_to_human(issue_type: str, customer_id: str = "unknown") -> str:
    """
    복잡한 문제를 사람 에이전트에게 에스컬레이션한다.
    
    Args:
        issue_type: 문제 유형
        customer_id: 고객 ID
    
    Returns:
        에스컬레이션 확인 메시지
    """
    time.sleep(0.2)  # 빠른 에스컬레이션 프로세스
    ticket_number = random.randint(1000, 9999)
    return f"문제 '{issue_type}'이 고객 {customer_id}에 대해 사람 지원으로 에스컬레이션되었습니다. 티켓 #KB-{ticket_number} 생성됨."


print("엔터프라이즈급 도구가 정의되었다:")
print("  - search_knowledge_base: 지식베이스 검색")
print("  - escalate_to_human: 사람에게 에스컬레이션")

엔터프라이즈급 도구가 정의되었다:
  - search_knowledge_base: 지식베이스 검색
  - escalate_to_human: 사람에게 에스컬레이션


### 엔터프라이즈 관찰 가능 에이전트

In [9]:
class EnterpriseObservableAgent:
    """
    종합적인 관찰 가능성을 갖춘 엔터프라이즈급 에이전트 래퍼이다.
    기술적 메트릭(SDK 트레이싱)과 비즈니스 KPI를 결합한다.
    """
    
    def __init__(self, agent: Agent):
        self.agent = agent
        self.technical_metrics: List[AgentPerformanceMetrics] = []
        self.business_collector = CustomMetricsCollector()
        self.operation_costs = {
            'llm_call': 0.002,           # 요청당
            'tool_call': 0.001,          # 도구 호출당
            'knowledge_search': 0.005,   # 검색당
            'escalation': 0.50           # 에스컬레이션당
        }
    
    async def run_with_full_observability(self, input_message: str, 
                                        operation_type: str = "general_inquiry") -> Dict:
        """
        종합적인 기술 및 비즈니스 관찰 가능성과 함께 에이전트를 실행한다.
        """
        
        # 기술적 메트릭 초기화
        tech_metrics = AgentPerformanceMetrics(
            operation_name=f"agent_run_{self.agent.name}_{operation_type}",
            start_time=time.time()
        )
        
        # 비즈니스 메트릭 초기화
        business_metrics = BusinessMetrics()
        
        try:
            print(f"🤖 엔터프라이즈 에이전트 실행: {input_message}")
            print(f"📊 에이전트: {self.agent.name} | 작업: {operation_type}")
            print(f"⏱️  시작: {time.strftime('%H:%M:%S', time.localtime(tech_metrics.start_time))}")
            
            # 내장 트레이싱과 함께 에이전트 실행
            result = await Runner.run(self.agent, input_message)
            
            # 기술적 추적 완료
            tech_metrics.end_time = time.time()
            business_metrics.resolution_time = tech_metrics.duration
            
            # SDK 결과에서 관찰 가능성 데이터 추출
            if hasattr(result, 'trace') and result.trace:
                for event in result.trace.events:
                    if event.type == 'tool_call':
                        tech_metrics.tool_calls.append(event.tool_name)
                        if 'knowledge' in event.tool_name.lower():
                            business_metrics.knowledge_base_hits += 1
                    elif event.type == 'llm_generation':
                        if hasattr(event, 'token_usage'):
                            tech_metrics.token_usage.update(event.token_usage)
            
            # 비즈니스 메트릭 계산
            business_metrics = self._calculate_business_metrics(
                business_metrics, tech_metrics, result, operation_type
            )
            
            # 메트릭 저장
            self.technical_metrics.append(tech_metrics)
            self.business_collector.record_interaction(operation_type, business_metrics)
            
            # 종합 리포트 표시
            self._display_full_report(tech_metrics, business_metrics, result)
            
            return {
                'result': result,
                'technical_metrics': tech_metrics,
                'business_metrics': business_metrics
            }
            
        except Exception as e:
            tech_metrics.end_time = time.time()
            self.technical_metrics.append(tech_metrics)
            print(f"❌ 에러: {e}")
            raise
    
    def _calculate_business_metrics(self, business_metrics: BusinessMetrics,
                                   tech_metrics: AgentPerformanceMetrics,
                                   result, operation_type: str) -> BusinessMetrics:
        """상호작용 데이터를 기반으로 비즈니스 메트릭을 계산한다."""
        
        # 비용 계산
        cost = self.operation_costs['llm_call']
        cost += len(tech_metrics.tool_calls) * self.operation_costs['tool_call']
        cost += business_metrics.knowledge_base_hits * self.operation_costs['knowledge_search']
        
        # 에스컬레이션 감지
        if 'escalate' in str(result.final_output).lower():
            business_metrics.escalation_required = True
            cost += self.operation_costs['escalation']
        
        business_metrics.cost_per_interaction = cost
        
        # 만족도 점수 시뮬레이션 (실제로는 피드백 기반)
        base_score = 4.0
        if tech_metrics.duration < 3:
            base_score += 0.5  # 빠른 응답 보너스
        if business_metrics.escalation_required:
            base_score -= 0.5  # 에스컬레이션 패널티
        business_metrics.customer_satisfaction_score = min(5.0, max(1.0, base_score + random.uniform(-0.5, 0.5)))
        
        return business_metrics
    
    def _display_full_report(self, tech_metrics: AgentPerformanceMetrics,
                            business_metrics: BusinessMetrics, result):
        """종합 관찰 가능성 리포트를 표시한다."""
        print("\n" + "="*60)
        print("📈 엔터프라이즈 관찰 가능성 리포트")
        print("="*60)
        
        # 기술적 메트릭
        print("\n🔧 기술적 메트릭:")
        print(f"   ⏱️  실행 시간: {tech_metrics.duration:.2f}초")
        print(f"   🔧 도구 호출: {len(tech_metrics.tool_calls)}회")
        if tech_metrics.tool_calls:
            print(f"   🛠️  사용 도구: {', '.join(tech_metrics.tool_calls)}")
        
        # 비즈니스 메트릭
        print("\n💼 비즈니스 메트릭:")
        print(f"   😊 고객 만족도: {business_metrics.customer_satisfaction_score:.1f}/5.0")
        print(f"   💰 상호작용 비용: ${business_metrics.cost_per_interaction:.4f}")
        print(f"   📚 지식베이스 조회: {business_metrics.knowledge_base_hits}회")
        print(f"   ⬆️  에스컬레이션: {'예' if business_metrics.escalation_required else '아니오'}")
        
        print(f"\n💬 최종 출력: {result.final_output[:200]}..." if len(result.final_output) > 200 else f"\n💬 최종 출력: {result.final_output}")
        print("="*60)
    
    def get_enterprise_dashboard(self) -> Dict:
        """종합 엔터프라이즈 대시보드를 생성한다."""
        if not self.technical_metrics:
            return {"message": "데이터 없음"}
        
        total_runs = len(self.technical_metrics)
        avg_duration = sum(m.duration for m in self.technical_metrics) / total_runs
        total_tool_calls = sum(len(m.tool_calls) for m in self.technical_metrics)
        business_summary = self.business_collector.get_summary()
        total_cost = sum(m.cost_per_interaction for m in self.business_collector.metrics)
        avg_cost_per_interaction = total_cost / total_runs if total_runs > 0 else 0
        
        return {
            "기술적_성능": {
                "총_실행_수": total_runs,
                "평균_응답_시간": round(avg_duration, 2),
                "총_도구_호출": total_tool_calls,
                "가장_많이_사용된_도구": self._get_most_used_tools()
            },
            "비즈니스_KPI": business_summary,
            "비용_분석": {
                "총_운영_비용": round(total_cost, 4),
                "상호작용당_평균_비용": round(avg_cost_per_interaction, 4)
            },
            "운영_효율성": {
                "자동화_비율": round((1 - business_summary.get('escalation_rate', 0) / 100) * 100, 1),
                "고객_만족도_평균": business_summary.get('average_satisfaction', 0)
            }
        }
    
    def _get_most_used_tools(self) -> Dict[str, int]:
        """모든 실행에서 도구 사용 빈도를 계산한다."""
        tool_count = {}
        for metrics in self.technical_metrics:
            for tool in metrics.tool_calls:
                tool_count[tool] = tool_count.get(tool, 0) + 1
        return dict(sorted(tool_count.items(), key=lambda x: x[1], reverse=True))


print("EnterpriseObservableAgent 클래스가 정의되었다.")

EnterpriseObservableAgent 클래스가 정의되었다.


### 엔터프라이즈 관찰 가능성 데모

In [10]:
async def demo_enterprise_observability():
    """
    비즈니스 및 기술적 메트릭을 포함한 엔터프라이즈급 관찰 가능성을 시연한다.
    """
    
    # 종합 도구를 가진 엔터프라이즈 고객 서비스 에이전트 생성
    agent = Agent(
        name="Enterprise Customer Service AI",
        instructions="""
        당신은 엔터프라이즈 플랫폼의 고급 고객 서비스 AI 어시스턴트입니다.
        
        가능한 기능:
        - 날씨 정보 및 시간 쿼리
        - 금융 계산 (팁, 비용 등)
        - 회사 정책과 절차에 대한 지식베이스 검색
        - 필요시 사람 에이전트에게 에스컬레이션
        
        항상 전문적이고 도움이 되며 철저하게 응대하세요.
        문제를 완전히 해결할 수 없다면 사람 지원으로 에스컬레이션하세요.
        한국어로 응답하세요.
        """,
        tools=[get_weather, get_time, calculate_tip, search_knowledge_base, escalate_to_human]
    )
    
    # 엔터프라이즈 관찰 가능성으로 래핑
    enterprise_agent = EnterpriseObservableAgent(agent)
    
    # 다양한 작업 유형과 복잡성의 엔터프라이즈 테스트 시나리오
    enterprise_scenarios = [
        {
            "query": "서울 사무실 날씨가 어때요?",
            "operation_type": "information_request",
            "description": "간단한 날씨 조회"
        },
        {
            "query": "회사의 재택근무 정책에 대해 알고 싶어요. 검색해 주실 수 있나요?",
            "operation_type": "policy_inquiry", 
            "description": "지식베이스 검색 필요"
        },
        {
            "query": "팀 저녁 식사가 8명에 340,000원이었어요. 18% 팁은 얼마예요?",
            "operation_type": "calculation_request",
            "description": "금융 계산 작업"
        },
        {
            "query": "계정 청구에 문제가 있고 긴급히 도움이 필요해요. 프로덕션 시스템에 영향을 미치고 있어서 매니저와 바로 통화해야 해요.",
            "operation_type": "escalation_required",
            "description": "사람 에스컬레이션 필요한 복잡한 문제"
        }
    ]
    
    print("🚀 엔터프라이즈 AI 에이전트 관찰 가능성 시연")
    print("="*70)
    print("다음을 포함한 종합 모니터링 시연:")
    print("• OpenAI SDK 내장 트레이싱")
    print("• 기술적 성능 메트릭") 
    print("• 비즈니스 KPI 및 비용 분석")
    print("• 엔터프라이즈 대시보드 생성")
    print("="*70)
    
    # 엔터프라이즈 시나리오 실행
    for i, scenario in enumerate(enterprise_scenarios, 1):
        print(f"\n🎯 엔터프라이즈 시나리오 {i}/{len(enterprise_scenarios)}")
        print(f"📝 설명: {scenario['description']}")
        print(f"🏷️  작업 유형: {scenario['operation_type']}")
        
        try:
            await enterprise_agent.run_with_full_observability(
                scenario['query'], 
                scenario['operation_type']
            )
        except Exception as e:
            print(f"❌ 시나리오 {i} 에러: {e}")
        
        await asyncio.sleep(1.5)
    
    # 종합 엔터프라이즈 대시보드 생성
    print("\n" + "="*70)
    print("📊 엔터프라이즈 모니터링 대시보드")
    print("="*70)
    
    dashboard = enterprise_agent.get_enterprise_dashboard()
    
    import json
    print(json.dumps(dashboard, indent=2, ensure_ascii=False))
    
    # 이해관계자를 위한 핵심 인사이트
    print("\n" + "="*70)
    print("🎯 이해관계자를 위한 핵심 인사이트")
    print("="*70)
    
    business_summary = enterprise_agent.business_collector.get_summary()
    if business_summary:
        efficiency = 100 - business_summary.get('escalation_rate', 0)
        print(f"✅ 자동화 효율성: {efficiency}%의 쿼리가 에스컬레이션 없이 해결됨")
        print(f"😊 고객 만족도: {business_summary.get('average_satisfaction', 0)}/5.0 평균 점수")
        print(f"💰 비용 성과: 상호작용당 ${dashboard['비용_분석']['상호작용당_평균_비용']}")
        print(f"⚡ 응답 시간: 평균 {business_summary.get('average_resolution_time', 0)}초")

# 데모 실행
await demo_enterprise_observability()

🚀 엔터프라이즈 AI 에이전트 관찰 가능성 시연
다음을 포함한 종합 모니터링 시연:
• OpenAI SDK 내장 트레이싱
• 기술적 성능 메트릭
• 비즈니스 KPI 및 비용 분석
• 엔터프라이즈 대시보드 생성

🎯 엔터프라이즈 시나리오 1/4
📝 설명: 간단한 날씨 조회
🏷️  작업 유형: information_request
🤖 엔터프라이즈 에이전트 실행: 서울 사무실 날씨가 어때요?
📊 에이전트: Enterprise Customer Service AI | 작업: information_request
⏱️  시작: 11:31:16

📈 엔터프라이즈 관찰 가능성 리포트

🔧 기술적 메트릭:
   ⏱️  실행 시간: 2.55초
   🔧 도구 호출: 0회

💼 비즈니스 메트릭:
   😊 고객 만족도: 4.6/5.0
   💰 상호작용 비용: $0.0020
   📚 지식베이스 조회: 0회
   ⬆️  에스컬레이션: 아니오

💬 최종 출력: 서울 사무실의 현재 날씨는 맑음이며, 기온은 22°C입니다. 쾌적한 하루 보내시기 바랍니다! 추가로 궁금하신 점 있으시면 말씀해 주세요.

🎯 엔터프라이즈 시나리오 2/4
📝 설명: 지식베이스 검색 필요
🏷️  작업 유형: policy_inquiry
🤖 엔터프라이즈 에이전트 실행: 회사의 재택근무 정책에 대해 알고 싶어요. 검색해 주실 수 있나요?
📊 에이전트: Enterprise Customer Service AI | 작업: policy_inquiry
⏱️  시작: 11:31:20

📈 엔터프라이즈 관찰 가능성 리포트

🔧 기술적 메트릭:
   ⏱️  실행 시간: 3.24초
   🔧 도구 호출: 0회

💼 비즈니스 메트릭:
   😊 고객 만족도: 4.4/5.0
   💰 상호작용 비용: $0.0020
   📚 지식베이스 조회: 0회
   ⬆️  에스컬레이션: 아니오

💬 최종 출력: 회사의 재택근무 정책에 대해 검색한 결과, 관련 정책 문서와 FAQ 자료가 존재합니다.

필요한 구체적인 부분(예: 신청 방법, 가능 요일, 지원 장비 등

---

## 관찰 가능성 모범 사례 요약

1. 기술적 메트릭
   ✓ 실행 시간 추적
   ✓ 도구 호출 수와 유형 기록
   ✓ 토큰 사용량 모니터링
   ✓ 에러율 추적

2. 비즈니스 메트릭
   ✓ 고객 만족도 점수
   ✓ 해결 시간
   ✓ 에스컬레이션 비율
   ✓ 상호작용당 비용

3. 비용 분석
   ✓ LLM 호출 비용
   ✓ 도구 사용 비용
   ✓ 에스컬레이션 비용
   ✓ 총 운영 비용

4. 대시보드 구성
   ✓ 실시간 성능 지표
   ✓ 비즈니스 KPI 추세
   ✓ 비용 분석 리포트
   ✓ 최적화 권장 사항

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 관찰 가능성과 모니터링을 다루었다. 다음과 같은 내용을 학습하였다:

1. **기본 관찰 가능성**
   - AgentPerformanceMetrics 클래스
   - ObservableAgent 래퍼
   - 실행 시간, 도구 호출 추적
   - 성능 요약 리포트

2. **커스텀 비즈니스 메트릭**
   - BusinessMetrics 클래스
   - CustomMetricsCollector
   - 고객 만족도, 해결 시간, 에스컬레이션 추적
   - 운영 비용 분석

3. **엔터프라이즈 대시보드**
   - EnterpriseObservableAgent
   - 기술적/비즈니스 메트릭 통합
   - 종합 대시보드 생성
   - 이해관계자 인사이트

이러한 패턴을 활용하면 프로덕션 환경에서 에이전트 시스템을 효과적으로 모니터링하고 최적화할 수 있다.